In [1]:
import sys
import cv2
import time
import numpy as np
import mediapipe as mp
import tensorflow as tf

from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtGui import QImage, QPixmap, QIcon
from PyQt5.QtCore import QThread, pyqtSignal, Qt
from PyQt5.QtWidgets import QApplication, QDialog, QLabel, QPushButton, QVBoxLayout, QFileDialog, QHBoxLayout


In [2]:
class mediapipe_pose:
    def __init__(self):
        self.mp_holistic = mp.solutions.holistic 
        self.mp_drawing = mp.solutions.drawing_utils
    def mediapipe_detection(self,image,model):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        image.flags.writeable = False                 
        results = model.process(image)                 
        image.flags.writeable = True                   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
        return image, results
    def draw_styled_landmarks(self,image, results):
        self.mp_drawing.draw_landmarks(image, results.pose_landmarks, self.mp_holistic.POSE_CONNECTIONS,
                                 self.mp_drawing.DrawingSpec(color=(112,112,112), thickness=2, circle_radius=1), 
                                 self.mp_drawing.DrawingSpec(color=(94,200,0), thickness=2, circle_radius=1)
                                 ) 
    def extract_keypoints(self,results):
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        return np.concatenate([pose])
    def BBox(self,image,results):
        xList,yList,bbox = [],[],[]
        if results.pose_landmarks:
            for id,land in enumerate(results.pose_landmarks.landmark):
                h,w,c = image.shape # high,weight,chanel with img
                cx = int(land.x *w)
                cy = int(land.y *h)
                xList.append(cx)
                yList.append(cy)
            xmin,xmax = min(xList),max(xList)
            ymin,ymax = min(yList),max(yList)
            bbox = xmin,ymin,xmax,ymax
        return bbox

# In[ ]:

In [3]:
class VideoThread(QThread):
    change_pixmap_signal = pyqtSignal(np.ndarray)
    frame_processed_signal = pyqtSignal(np.ndarray)

    def __init__(self, video_source=0):
        super(VideoThread, self).__init__()
        self.video_source = video_source
        self.run_video = True

    def run(self):
        cap = cv2.VideoCapture(self.video_source)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
        while cap.isOpened() and self.run_video:
            ret, frame = cap.read()
            if ret:
                self.change_pixmap_signal.emit(frame)
                self.frame_processed_signal.emit(frame)
                time.sleep(0.033)
            else:
                break
        cap.release()

    def stop(self):
        self.run_video = False

class Ui_Dialog(QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Activity Recognition")
        self.setGeometry(100, 100, 800, 600)
        self.model = tf.keras.models.load_model('weight/LRCN_model.h5')

        self.main_layout = QVBoxLayout()
        self.video_label = QLabel(self)
        self.video_label.setFixedSize(640, 480)
        self.video_label.setStyleSheet("border: 1px solid black;")
        
        self.control_layout = QHBoxLayout()
        self.text_label = QLabel('Activity: ', self)
        self.text_label.setStyleSheet("font-size: 16px; font-weight: bold;")
        self.fps_label = QLabel('FPS: ', self)
        self.fps_label.setStyleSheet("font-size: 16px; font-weight: bold;")

        self.start_button = QPushButton('Start Camera', self)
        self.start_button.setIcon(QIcon('image/start_icon.png'))
        self.start_button.setStyleSheet("background-color: white; color: black; font-size: 14px;")
        self.start_button.clicked.connect(self.start_camera)

        self.load_video_button = QPushButton('Load Video', self)
        self.load_video_button.setIcon(QIcon('image/video_icon.png'))
        self.load_video_button.setStyleSheet("background-color: white; color: black; font-size: 14px;")
        self.load_video_button.clicked.connect(self.load_video)

        self.control_layout.addWidget(self.start_button)
        self.control_layout.addWidget(self.load_video_button)
        self.control_layout.addWidget(self.text_label)
        self.control_layout.addWidget(self.fps_label)

        self.main_layout.addWidget(self.video_label)
        self.main_layout.addLayout(self.control_layout)
        self.setLayout(self.main_layout)

        self.sequence = []
        self.actions = ['ApplyEyeMakeup','Basketball','Biking', 'BoxingPunchingBag','Eating', 'HorseRiding','Hugging','PlayingGuitar', 'Rafting','Salat','SalsaSpin','Waving', 'WritingOnBoard']
        self.thread = None
        self.pTime = time.time()

    def start_camera(self):
        if self.thread is not None:
            self.thread.stop()
        self.thread = VideoThread()
        self.thread.change_pixmap_signal.connect(self.update_display)
        self.thread.frame_processed_signal.connect(self.process_frame)
        self.thread.start()

    def load_video(self):
        path, _ = QFileDialog.getOpenFileName(self, "Open Video", "", "Video Files (*.mp4 *.flv *.ts *.mts *.avi)")
        if path:
            if self.thread is not None:
                self.thread.stop()
            self.thread = VideoThread(video_source=path)
            self.thread.change_pixmap_signal.connect(self.update_display)
            self.thread.frame_processed_signal.connect(self.process_frame)
            self.thread.start()

    @QtCore.pyqtSlot(np.ndarray)
    def update_display(self, frame):
        """ Update video label to show new frame without unnecessary resizing. """
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = frame_rgb.shape
        bytes_per_line = ch * w
        convert_to_Qt_format = QImage(frame_rgb.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(convert_to_Qt_format)
        self.video_label.setPixmap(pixmap.scaled(self.video_label.width(), self.video_label.height(), Qt.KeepAspectRatio))

    @QtCore.pyqtSlot(np.ndarray)
    def process_frame(self, frame):
        """ Process each frame for activity recognition. """
        frame_resized = cv2.resize(frame, (64, 64), interpolation=cv2.INTER_AREA)
        frame_normalized = frame_resized / 255.0
        self.sequence.append(frame_normalized)
        if len(self.sequence) == 20:
            self.sequence = np.array(self.sequence)
            self.sequence = self.sequence[np.newaxis, ...]
            predictions = self.model.predict(self.sequence)
            predicted_action = self.actions[np.argmax(predictions)]
            self.text_label.setText('Activity: ' + predicted_action)
            self.sequence = []
        cTime = time.time()
        time_diff = cTime - self.pTime
        if time_diff > 0:
            fps = 1 / time_diff
            self.fps_label.setText(f"FPS: {int(fps)}")
        self.pTime = cTime

if __name__ == "__main__":
    app = QApplication(sys.argv)
    mainWin = Ui_Dialog()
    mainWin.show()
    sys.exit(app.exec_())


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━

SystemExit: 0

c:\Users\oumai\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
